# The Log Likelihood Problem for a Mixture of Multinomials

Preamble:
There are three things that we have to address before we can proceed with this

1. Why is the multinomial different to a sequence of draws from a categorical, and what is the impact of that
2. What is the Polya distribution, anyway.
3. What is the difference between log likelihood and predictive log likelihood

# The two approaches to a Mixture of Multinomials

Consider the word 'MISSISSIPPI'. The probability of the first letter -- assuming just 26 characters in the (Latin) alphabet -- is 1/26, and so on for the rest of the letters, so it's essentially

$$\text{Pr}(X = {\tt{'MISSISSIPPI'}}) = \prod_i \frac{1}{26}$$

If letters had differing probabilities $\phi_1, \ldots \phi_26$ then we would have

$$\text{Pr}(X = {\tt{'MISSISSIPPI'}}) = \prod_i \prod_l \phi_{l}^{x_{il}}$$

where $x_{il}$ is a 1-of-26 indicator vector which indicates which letter $l$ is observed for which character position $i$. For example, the third letter is 'S', which is the 19th letter of the alphabet, so $x_{3,19} = 1$ and $x_{3,l \neq 19} = 0$

Assume instead that we represent this is a bag of words vector, i.e. counts of each individual letters. We will have $\{{\tt{'M'}}:1, {\tt{'I'}}:4, {\tt{'S'}}: 4, {\tt{'P'}}: 2 \}$ and zero for everything else. The number of 11-letter words we can obtain by the formula is given by the permuation of multisets formula (i.e. the multinomial coefficient) $\frac{11!}{1! 4! 4! 2!}$. Thus if we're given letter-counts like this as a representation of an obsevation, and do not know the actual observed _ordering_ in addition to letter counts, we have to count all possible orderings, of which there will be $\frac{11!}{1! 4! 4! 2!}$ possibilities. More generally, for a word-count vector $\boldsymbol{x} \in \mathbb{N}^{26}$ our probability will be

$$\text{Pr}(X = \boldsymbol{x}) = \frac{(\sum_l x_l)!}{\prod_l x_l!} \prod_l \phi_l ^{x_l}$$

which we recognise as the PMF of a multinomial distribution.

So the difference between the two PMFs -- the joint probability of an observed sequence of letters, or the probability of a all possible sequence of a letter mix -- depends only on whether or not the ordering is observed. Which choice you make has no effect on parameter estimation, since $\phi$ does not appear in the multinomial coefficient. It _will_ affect estimates of the probability of a particular observation given a parameter estimate $\hat{\phi}$. 

It does appear from this that if we assume letters in a word are exchangeable (or as a more pertinent example, words within a document are exchangeable), then the Multinomial is the only valid choice. Why this is not the case is FIXME

Well, the reality is that we _see_ the particular ordering. But what about `"The way the cat went"`. There's two `"the"`s there. Isn't that really _two_ sequences? Well, recall the binomial coefficient is a _combination_ and not a _permutation_. The multinomial coefficient similarly does... FIXME

# The Polya Distribution 

See Wikipedia. Note it inherits $\frac{n!}{x_i!}$ terms from the embedded multinomial. If we observe the sequence ordering, these disappear.

# Posterior Predictive Probability and the Likelihood

It's good to consider the use-case. 

1. For ML:
 * You consider the probability of training data given the parameter value $p(\mathcal{X};\phi)$ and find the value of that $\phi$ that maximises it. The posterior predictive is then $p(x;\hat{\phi})$.
 * In the latent case, you have to integrate out the possible settings of $z$, i.e. $p(x;\hat{\phi}) = \int_z p(x, z;\hat{\phi}) = \int_z p(x;\hat{\phi})p(z) = \sum_k \theta_k p(x;\phi_k)$ where $\theta_k$ is the _prior_ probability of $z$
2. For MAP
 * You consider the probability of training data given the parameter value and its prior $p(\mathcal{X}|\phi)p(\phi)$, but ultimately the posterior predictive is still $p(x|\hat{\phi})$.
 * And so in the latent case it's still the same
3. For Bayes
 * You evaluate $p(\phi|\mathcal{X}) = \frac{p(x|\phi)p(\phi)}{\int_\phi p(x|\phi)p(\phi)}$
 * Your posterior predictive is then $p(x|\mathcal{X}) = \int_\phi p(x|\phi)p(\phi|\mathcal{X})$
     * Options then are to solve for $p(\phi|\mathcal{X})$ analytically, and then analytically solve the integral
     * Derive an approximation $q(\phi) \approx p(\phi|\mathcal{X})$ and then use it to analytically solve the integral $p(x|\mathcal{X}) = \int_\phi p(x|\phi)p(\phi|\mathcal{X}) \approx \int_\phi p(x|\phi)q(\phi)$
     * Or draw samples from $p(\phi|\mathcal{X})$ (perhaps via an approximate method like Metroplois-Hasting) and then approximate the integral as $p(x|\mathcal{X}) \approx \frac{1}{S}\sum_s p(x|\phi_s)$


The important point is that in the fully Bayesian case it doesn't make a lot of sense to start thinking about the log-likelihood of the training data _first_. It makes a lot more sense to think about the posterior-predictive likelihood of the unseen data, and work back.

In the case above for mixture of multinomials we want


$$
p(x|\mathcal{X}) = \int_z \int_\phi \int_\theta p(x|z, \phi) p(z|\theta) p(\theta, \phi|\mathcal{X})
$$

where we've already marginalized out $\mathcal{Z}$ from our parameter posterior $p(\theta, \phi|\mathcal{X}) = \int_{\mathcal{Z}} p(\theta, \phi, \mathcal{Z}|\mathcal{X})$

If we use mean-field variational, then 

$$
p(\theta, \phi, \mathcal{Z}|\mathcal{X}) \approx q(\theta)q(\phi)q(\mathcal{Z})
$$
And so the marginalised parameter posterior is just $q(\theta)q(\phi)$

And the posterior predictive is then

$$
\begin{align*}
p(x|\mathcal{X}) 
& \approx \int_z \int_\theta \int_\phi p(x|z, \phi) p(z|\theta)q(\theta)q(\phi) \\
& = \sum_k \int_\theta \int_{\phi_k} p(x|\phi_k) \theta_k q(\theta)q(\phi_k)
\end{align*}
$$
Which works out as a product of Polyas iff you include the unknown ordering coefficient, and something similar to Polya if you don't.

Note in the second line that $q(\phi) = q(\phi_1), \ldots, q(\phi_K)$ (mean-field) and that $\mathbb{E}_Y[f(X)] = f(X)$ if $X$ is independent of $Y$. In this case $f(\phi_k)$ is independent of $\phi_j$ for all $j \neq k$ according to our factoried approximation.